In [11]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")
import numpy as np
import matplotlib.pyplot as plt

import dill
from sklearn.mixture import BayesianGaussianMixture
from data import get_data
from sklearn.metrics import roc_curve, auc
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from utils import BEST_SCORES


In [12]:
# Need to load all the data
def return_adam_files(data_name):
    files = [
        f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:{i}_end:{i+20}.p"
        for i in np.linspace(0, 280, 15, dtype=int)
    ]
    return files


def return_bfgs_files(data_name):
    files = [
        f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:{i}_end:{i+20}.p"
        for i in np.linspace(0, 280, 15, dtype=int)
    ]
    return files

In [13]:
def convert_file_scores_into_dict(files):
    """
    Convert files saved by runs to one big dict of scores with the key being
    the run_number and the value being the tuple of a tuple of scores of the
    form ( (x, y|x), (y, x|y) )
    """
    work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
    all_scores = {}
    for file_idx in range(len(files)):
        # Open  file results
        with open(f"{work_dir}/results/{files[file_idx]}", "rb") as f:
            results = dill.load(f)
        for i in range(len(results['scores'])):
            idx = 10 * file_idx + i
            all_scores[idx] = results["scores"][i]
    return all_scores

In [14]:
def return_best_between_two_scores(scores_1, scores_2):
    """
    Scores should be dict with values of tuples of tuples
    ( (x, y|x), (y, x|y) )
    """
    all_scores = {}
    for idx in scores_1.keys(): 
        if idx not in scores_2.keys():
            raise ValueError(f"Run idx mismatch for run {idx}")
        else:
            scores_1_idx = scores_1[idx]
            scores_2_idx = scores_2[idx]
            min_score_x = min(scores_1_idx[0][0], scores_2_idx[0][0])
            min_score_y_x = min(scores_1_idx[0][1], scores_2_idx[0][1]) 
            min_score_y = min(scores_1_idx[1][0], scores_2_idx[1][0])
            min_score_x_y = min(scores_1_idx[1][1], scores_2_idx[1][1]) 
            final_scores = (
                (min_score_x, min_score_y_x),
                (min_score_y, min_score_x_y)
            )
            all_scores[idx] = final_scores
    return all_scores

In [15]:
def return_best_between_marginal_and_full_scores(marginal_score, full_score):
    """
    Scores should be dict with values of tuples of tuples
    maringal_score = (x, y)
    full_score: ( (x, y|x), (y, x|y) )
    """
    all_scores = {}
    for idx in marginal_score.keys(): 
        if idx not in full_score.keys():
            pass
            # raise ValueError(f"Run idx mismatch for run {idx}")
        else:
            scores_marg_idx = marginal_score[idx]
            scores_full_idx = full_score[idx]
            # min_score_x = min(scores_marg_idx[0], scores_full_idx[0][0])
            min_score_x = scores_marg_idx[0]
            min_score_y_x = scores_full_idx[0][1] 
            # min_score_y = min(scores_marg_idx[1], scores_full_idx[1][0])
            min_score_y =  scores_marg_idx[1] 
            min_score_x_y = scores_full_idx[1][1]
            final_scores = (
                (min_score_x, min_score_y_x),
                (min_score_y, min_score_x_y)
            )
            all_scores[idx] = final_scores
    return all_scores

In [16]:
def return_best_between_marginal_and_full_scores_namedtuple(marginal_score, full_score):
    """
    Scores should be dict with values of named_tuple
    maringal_score = (x, y)
    full_score: ( (x, y|x), (y, x|y) )
    """
    all_scores = {}
    for idx in marginal_score.keys(): 
        if idx not in full_score.keys():
            pass
            # raise ValueError(f"Run idx mismatch for run {idx}")
        else:
            scores_marg_idx = marginal_score[idx]
            scores_full_idx = full_score[idx]
            min_score_x = min(scores_marg_idx[0], scores_full_idx.best_loss_x)
            # min_score_x = scores_marg_idx[0]
            # min_score_x = scores_full_idx.best_loss_x
            min_score_y_x = scores_full_idx.best_loss_y_x 
            min_score_y = min(scores_marg_idx[1], scores_full_idx.best_loss_y)
            # min_score_y =  scores_marg_idx[1] 
            # min_score_y = scores_full_idx.best_loss_y
            min_score_x_y = scores_full_idx.best_loss_x_y
            final_scores = BEST_SCORES(
                min_score_x, min_score_y_x,
                min_score_y, min_score_x_y 
            )
            all_scores[idx] = final_scores
    return all_scores

In [17]:
def get_auc_scores(data_name, scores, randomise=False):
    work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
    data_get = getattr(get_data, f"get_tubingen_pairs_dataset")
    x, y, weight, target = data_get(data_path=f"{work_dir}/data/pairs/files")

    y_scores = []
    y_labels = []
    for idx in scores.keys():
        causal = sum(scores[idx][0])
        anti_causal = sum(scores[idx][1])
        final_score = - causal + anti_causal 
        y_labels.append(target[idx])
        y_scores.append(final_score)

    if randomise:
        random_choice = np.random.choice(len(y_labels), size=len(y_labels) // 2)
        for i in random_choice:
            y_labels[i] *= -1
            y_scores[i] *= -1

    fpr, tpr, _ = roc_curve(y_labels, y_scores)
    roc_auc = auc(fpr, tpr)
    return roc_auc


In [18]:
def get_scores(best_scores):
    total_runs = len(list(best_scores.keys()))
    y_scores = {}
    y_scores_array = np.zeros(total_runs)
    for idx, run_idx in enumerate(list(best_scores.keys())):
        current_run = best_scores[run_idx]
        causal_score = current_run.best_loss_x + current_run.best_loss_y_x
        anticausal_score = current_run.best_loss_y + current_run.best_loss_x_y
        y_scores[run_idx] = - causal_score + anticausal_score
        y_scores_array[idx] = - causal_score + anticausal_score 
    return y_scores, y_scores_array

In [9]:
data_name = "cha_pairs"
# adam_files = return_adam_files(data_name=data_name)
bfgs_files = return_bfgs_files(data_name=data_name)
# adam_scores = convert_file_scores_into_dict(adam_files)
bfgs_scores = convert_file_scores_into_dict(bfgs_files)
# scores = return_best_between_two_scores(adam_scores, bfgs_scores)
adam_bfgs_auc = get_auc_scores(data_name, bfgs_scores)

KeyboardInterrupt: 

In [19]:
data_name = "cha_pairs"
bfgs_files = return_bfgs_files(data_name=data_name)

work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"

all_results = {}
for file in bfgs_files:
    with open(f"{work_dir}/results/{file}", "rb") as f:
        results = dill.load(f)
    all_results.update(results["final_scores"])
    
from utils import return_all_scores, return_best_causal_scores, get_correct


all_x, all_y_x, all_y, all_x_y = return_all_scores(all_results)
best_scores = return_best_causal_scores(all_x, all_y_x, all_y, all_x_y)

y_scores_dict, y_scores = get_scores(best_scores)


work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
data_get = getattr(get_data, f"get_{data_name}_dataset")
x, y, weight, target = data_get(data_path=f"{work_dir}/data/cha_pairs/files")

from sklearn.metrics import roc_auc_score
gplvm_roc_auc = roc_auc_score(target, y_scores)

2023-01-05 11:21:01.009049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 11:21:01.192348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 11:21:01.194002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 11:21:01.195981: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [20]:
gplvm_roc_auc

0.8084983314794216

In [21]:
def bayesgmm_score(train_data, n_components):
    model = BayesianGaussianMixture(
        n_components=n_components,
        max_iter=int(1e6),
    ).fit(train_data)
    return - np.sum(model.score_samples(train_data))
    # return - model.lower_bound_

In [26]:
def return_bayesian_gmm_scores(x, y, num_restarts):
    """
    Will return scores of Bayesian GMM for a dataset.
    """
    all_scores = {}
    n_components_array = np.arange(3, 20)
    for idx in tqdm(range(len(x)), desc="Running BayesGMM"):
        run_score_x = []
        run_score_y = []
        for i in range(num_restarts):
            train_x = x[idx]
            train_y = y[idx]
            # Normalise the data
            train_x = StandardScaler().fit_transform(train_x).astype(np.float64)
            train_y = StandardScaler().fit_transform(train_y).astype(np.float64)

            n_components = np.random.choice(n_components_array)
            x_score = bayesgmm_score(
                train_x,
                n_components=n_components,
            )
            y_score = bayesgmm_score(
                train_y,
                n_components=n_components,
            )
            
            run_score_x.append(x_score)
            run_score_y.append(y_score)
     
        all_scores[idx] = (min(run_score_x), min(run_score_y))
    return all_scores

In [27]:
bayesgmm_scores = return_bayesian_gmm_scores(x, y, 20)

Running BayesGMM:  14%|█▎        | 41/300 [29:10<3:21:13, 46.61s/it]

In [24]:
best_gmm_full_scores = return_best_between_marginal_and_full_scores_namedtuple(
    bayesgmm_scores, best_scores 
)
y_scores_dict, y_scores_new = get_scores(best_gmm_full_scores)

work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
data_get = getattr(get_data, f"get_{data_name}_dataset")
x, y, weight, target = data_get(data_path=f"{work_dir}/data/cha_pairs/files")

from sklearn.metrics import roc_auc_score
gmm_roc_auc = roc_auc_score(target, y_scores_new)

In [25]:
gmm_roc_auc

0.7414905450500555

In [94]:
target

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
      

In [95]:
y_scores

array([ 3.09154188e+02,  3.66191873e+02,  4.49758413e+02,  5.14750642e+00,
        2.49209975e+01, -7.82399492e+01, -1.19024906e+02,  6.09052583e+00,
        4.72481347e+01,  1.05600857e+02, -1.76928503e+00,  1.83276450e+02,
        5.80673366e+01,  3.58231556e+01, -3.91581030e+01, -4.08265993e+01,
        2.22869822e+01,  1.85391849e+01,  1.91701252e+01, -2.05835366e+01,
        7.99596331e+01, -5.60123450e+01,  2.49699407e+01,  3.28778441e+01,
       -2.66226931e+02,  2.31138341e+00, -2.80871160e+02, -1.13328534e+02,
       -3.61919793e+01,  2.25463516e+01,  4.20985027e+02, -1.21264004e+01,
        1.71918069e+01,  3.93022020e+02,  1.22471599e+02,  1.20581055e+01,
       -3.84801918e+02,  2.70032267e+01, -9.55158322e+00,  1.35954938e+01,
        3.86056627e+00,  7.53701291e+01, -6.69951180e+02,  1.08484600e+02,
        3.90585389e+00, -3.13215406e+03, -1.94747322e+03, -4.21791957e+01,
       -3.24376008e+01,  1.78044124e+03,  7.33877086e+01, -2.10842731e+02,
       -2.81576731e+02, -

In [96]:
y_scores_new

array([ 2.28464849e+02,  3.35808711e+02,  3.67637481e+02,  3.89965045e+00,
        4.75726429e+01, -2.60338075e+01, -6.11551819e+01,  5.44586224e+01,
        3.99848029e+00,  2.12149896e+01,  2.16965029e+01,  7.91717696e+01,
        7.25590961e+01,  6.78836008e+00, -7.29861271e+01, -6.58091966e+01,
        3.22520125e+00,  1.58802682e+01,  3.82745334e+01,  9.96602191e+00,
        5.82536467e+01, -2.97923424e+01,  2.19044486e+00,  7.28163155e+01,
       -1.80783312e+02,  2.76377638e+01, -2.79497826e+02, -5.91357995e+01,
        1.54568195e-01,  1.01552058e+02,  4.10705657e+02, -1.14814983e+01,
        4.47122639e+01,  3.56382184e+02,  1.21758325e+02,  8.64615882e+00,
       -5.58260303e+02, -1.39620246e+01, -1.92829759e+01, -1.06664708e+01,
       -1.43452419e+01,  5.61647824e+01, -6.24838948e+02,  5.58158084e+01,
       -3.17234512e+01, -3.32562817e+03, -1.88066912e+03, -6.36947829e+00,
       -1.45934657e+01,  1.71970482e+03,  1.33090625e+02, -1.99361876e+02,
       -2.49516777e+02,  

In [109]:
y_pred = np.zeros_like(y_scores)
for i in range(len(y_pred)):
    if y_scores[i] < 0:
        y_pred[i] = -1
    else:
        y_pred[i] = 1

In [112]:
target = np.ones(99)

In [116]:
(target == y_pred)

/tmp/ipykernel_1179361/2611233335.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (target == y_pred)


False

In [54]:
best_gmm_full_scores = return_best_between_marginal_and_full_scores(
    bayesgmm_scores, bfgs_scores 
)


NameError: name 'bfgs_scores' is not defined

In [51]:
best_gmm_full_scores

{0: ((1395.5631901180113, 930.9698778836593),
  (1729.0258240999706, 821.6612760377509)),
 1: ((2008.5887789291737, 883.4055007092063),
  (1886.5540574438267, 1341.2489331303461)),
 2: ((-161.22569620652746, 853.660222276736),
  (1358.08056782533, -369.76414350167863)),
 3: ((2076.117446438576, 1395.4239323475567),
  (1947.4141942581148, 1528.026834973469)),
 4: ((1788.2149422057523, 1754.2965052038862),
  (1980.9849244342004, 1609.0991658317469)),
 5: ((950.899970293185, 1781.7792980583624),
  (2114.5382929474645, 592.1071679461932)),
 6: ((1212.6791372188864, 455.23872672814457),
  (1350.5027629328765, 256.25991911622964)),
 7: ((544.6212171664836, 1303.9244484219332),
  (1677.681983046437, 225.32230489742813)),
 8: ((2123.003744013443, 1337.7132393866307),
  (1825.5021987585803, 1639.2132649329978)),
 9: ((-915.6970162418634, 677.1503258588157),
  (1459.4543063960364, -2362.774620434462)),
 10: ((2094.937383849984, 1609.1079366051767),
  (2127.5437298117677, 1598.1980935239264)),
 1

In [94]:
print(f"AUC: {adam_bfgs_auc}, GMM AUC: {bayes_gmm_auc}")

AUC: 0.6986817325800376, GMM AUC: 0.515426497277677
